In [1]:
import pandas as pd # standard naming convention
import numpy as np

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a pair of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every 'x' BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [2]:
#If haven't downloaded it yet, please get the data file with wget
#!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./data/

1\. Create a Pandas DataFrame reading N rows of the `data/data_000637.txt` dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k (check the documentation).

In [12]:
file_name = "./data/data_000637.txt"
data = pd.read_csv(file_name)

N=15000

data_N=data.head(N)

data_N

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
...,...,...,...,...,...,...
14995,1,1,4,3869200316,3399,9
14996,1,1,17,3869200316,3400,15
14997,1,1,10,3869200316,3530,16
14998,1,1,8,3869200316,3533,18


2\. Estimate the number of BX in a ORBIT (the value 'x').

In [13]:
x=[]
ind=0
for i in range(0,N-1):
    if(data_N['ORBIT_CNT'][i]==data_N['ORBIT_CNT'][i+1]):
        ind+=1
    else:
        x.append(data_N['BX_COUNTER'][i]-data_N['BX_COUNTER'][i-ind])
        ind=0      
x_est=np.mean(x)
print("Estimation: ",x_est)

x_est=max(data_N['BX_COUNTER'])+1
print("Estimation: ",x_est)

Estimation:  2489.6198830409357
Estimation:  3564


3\. Find out the duration of the data taking in hours, minutes and seconds. You can either make an estimate based on the fraction of the measurements (rows) you read, or perform this check precisely by reading the whole dataset.

In [14]:
time=[]
    
time.append(data_N['TDC_MEAS'][0]*(25/30))

for i in range(1,N):
    if(data_N['ORBIT_CNT'][i]==data_N['ORBIT_CNT'][i-1]):
        if(data_N['BX_COUNTER'][i]==data_N['BX_COUNTER'][i-1]):
            tdc=data_N['TDC_MEAS'][i]-data_N['TDC_MEAS'][i-1]
            time.append(tdc*(25/30))
        else:
            bx=data_N['BX_COUNTER'][i]-data_N['BX_COUNTER'][i-1]
            tdc=bx*30+data_N['TDC_MEAS'][i]-data_N['TDC_MEAS'][i-1]
            time.append(tdc*(25/30))
    else:
        cnt=(data_N['ORBIT_CNT'][i]-data_N['ORBIT_CNT'][i-1])
        bx=cnt*x_est+data_N['BX_COUNTER'][i]-data_N['BX_COUNTER'][i-1]
        tdc=bx*30+data_N['TDC_MEAS'][i]-data_N['TDC_MEAS'][i-1]
        time.append(tdc*(25/30))

time_s=np.array(time)*10**-9

tot_s=np.sum(time_s)
tot_m=tot_s/60
tot_h=tot_m/60
print("Estimation second",tot_s)
print("Estimation minute",tot_m)
print("Estimation hours",tot_h)


#another method


cnt1=(data_N['ORBIT_CNT'][N-1]-data_N['ORBIT_CNT'][0])
bx1=cnt1*x_est+data_N['BX_COUNTER'][N-1]-data_N['BX_COUNTER'][0]
tdc1=bx1*30+data_N['TDC_MEAS'][N-1]-data_N['TDC_MEAS'][0]
TT=tdc1*(25/30)*10**-9
tot_m1=TT/60
tot_h1=tot_m1/60
print("new method")
print("Estimation second",TT)
print("Estimation minute",tot_m1)
print("Estimation hours",tot_h1)












Estimation second 0.013305025000000002
Estimation minute 0.0002217504166666667
Estimation hours 3.695840277777778e-06
new method
Estimation second 0.013305003333333334
Estimation minute 0.00022175005555555557
Estimation hours 3.6958342592592595e-06


4\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information) since the beginning of the data acquisition.

In [17]:
data_N['TIME']=[np.sum(time[0:i]) for i in range(1,N+1)]
data_N

<ipython-input-17-7d4d803d6617>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_N['TIME']=[np.sum(time[0:i]) for i in range(1,N+1)]


,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,TIME
0,1,0,123,3869200167,2374,26,2.166667e+01
1,1,0,124,3869200167,2374,27,2.250000e+01
2,1,0,63,3869200167,2553,28,4.498333e+03
3,1,0,64,3869200167,2558,19,4.615833e+03
4,1,0,64,3869200167,2760,25,9.670833e+03
...,...,...,...,...,...,...,...
14995,1,1,4,3869200316,3399,9,1.330153e+07
14996,1,1,17,3869200316,3400,15,1.330156e+07
14997,1,1,10,3869200316,3530,16,1.330481e+07
14998,1,1,8,3869200316,3533,18,1.330489e+07


5\. Use the `.groupby()` method to find out the noisy channels, i.e. the TDC channels with most counts (print to screen the top 3 and the corresponding counts)

In [37]:
max_TDC=data_N.groupby(['TDC_CHANNEL']).max()
print(max_TDC[len(max_TDC):len(max_TDC)-4:-1])

             HEAD  FPGA   ORBIT_CNT  BX_COUNTER  TDC_MEAS          TIME
TDC_CHANNEL                                                            
139             1     1  3869200168        3186         0  30325.000000
124             1     0  3869200167        2374        27     22.500000
123             1     0  3869200167        2374        26     21.666667


6\. Count the number of non-empty orbits (i.e. the number of orbits with at least one hit). Count also the number of unique orbits with at least one measurement from TDC_CHANNEL=139.

7\. **Optional:** Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel